In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("You are an useful assistant to a software Engineer"),
    HumanMessage("Help me learn git")
]

llm.invoke(messages)

AIMessage(content='Okay, let\'s break down how to learn Git effectively. I\'ll provide a structured approach, covering the core concepts, essential commands, and practical tips.\n\n**1. Understanding the Core Concepts**\n\n*   **What is Git?** Git is a distributed version control system.  It tracks changes to files over time, allowing you to revert to specific versions, collaborate with others, and manage different branches of your project.\n*   **Repository (Repo):**  A repository is the main container for your project, storing all the files, history, and metadata.  It can be local (on your computer) or remote (on a server like GitHub, GitLab, or Bitbucket).\n*   **Commit:** A commit is a snapshot of your project at a specific point in time. It includes the changes you\'ve made since the last commit, along with a message describing those changes.  Commits are the building blocks of your project\'s history.\n*   **Branch:** A branch is a separate line of development. It allows you to w

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vectorStore = InMemoryVectorStore(embeddings)

In [6]:
vectorStore

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader


bs4_strainer = bs4.SoupStrainer(
    class_ = ( "post-title", "post-header", "post-content" )
)

loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={
        "parse_only": bs4_strainer
    }
)
docs = loader.load()
docs[0].page_content[:100]


USER_AGENT environment variable not set, consider setting it to identify your requests.


'\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |'

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_Splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True
)

all_splits = text_Splitter.split_documents(docs)

all_splits

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from

In [9]:
document_ids = vectorStore.add_documents(documents=all_splits)
document_ids

['734d887c-c46c-4355-9343-0b166c129847',
 '8742bd5b-6f19-4c7f-bf9c-08b9b1bce653',
 '3608cab1-12bd-4b3e-b4a0-0b79e48d0840',
 '43010385-2d4b-43f7-9050-086dd9587917',
 '5ec938c8-3ccc-4949-937b-f057887a9050',
 '739eb9ac-59eb-4c00-af5d-f6fb8fd913e7',
 '5cb3ba91-6f76-4f64-8766-ca1e6831e744',
 'a55216da-a202-408f-be7b-e6b996ceaf76',
 '61ade5eb-cec0-4dd1-b7e4-9bc707571b43',
 '5fcd7907-23a1-480b-bb24-ccb7812e77e3',
 '181fb45f-d36f-4c5a-936a-430c80147d09',
 '80fd3c18-bb8c-4dd5-8dc6-bdfe2217c74d',
 '8d70ae9a-1566-4295-a3e6-3a462113597e',
 'cb64d7b5-58b5-4855-971f-28170c99d6ab',
 'ae6f8436-b5a8-45fc-a85b-177149d969ef',
 '04aeb61d-bda4-4372-9f7c-b8386916c506',
 '49eb924b-3911-4ae3-9cef-e19cb1582505',
 'b55bcbc2-93bf-415a-a0af-88aca3f24cc2',
 '9f1980b3-c69a-40a3-b1ad-caf8c36237ac',
 'f8062865-8de3-4f14-91a4-f8e2db4455a8',
 '3fb3f53f-5838-4c92-a9c2-4d3add106384',
 '9f408aa6-ef2a-4760-8207-ccaea746912b',
 '744c08f1-3d27-48a6-b210-3628a960efbb',
 '86b53e78-66f8-4339-8322-18f153bcd72f',
 'd9e87092-aa37-

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_message = prompt.invoke(
    {
        "context": "You are an assistant",
        "question" : "summarize the content"
    }
)

example_message

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: summarize the content \nContext: You are an assistant \nAnswer:", additional_kwargs={}, response_metadata={})])

In [11]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class State(TypedDict):
    question: str
    context: List[Document]
    answer : str
    

In [12]:
def retrieve(state: State):
    retrieved_docs = vectorStore.similarity_search(state["question"])
    return {
        "context": retrieved_docs
    }

def generate(state: State):
    docs_content = "\n\n".join( doc.page_content for doc in state["context"] )
    message = prompt.invoke({
        "question" : state["question"],
        "context" : docs_content
    })
    response = llm.invoke(message)
    return {
        "answer": response.content
    }


In [13]:
from langgraph.graph import START, StateGraph

graphBuilder = StateGraph(State).add_sequence([retrieve, generate])
graphBuilder.add_edge(START, "retrieve")
graph = graphBuilder.compile()

In [14]:
result = graph.invoke({
    "question" : "What is  task decomposotion",
})
result


{'question': 'What is  task decomposotion',
 'context': [Document(id='3608cab1-12bd-4b3e-b4a0-0b79e48d0840', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1638}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree st

In [17]:
generate.__call__

<method-wrapper '__call__' of function object at 0x00000281EDD24040>